In [1]:
# импортируем необходимые библиотеки, классы и функции
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split, 
                                     GridSearchCV)
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (PowerTransformer, 
                                   OneHotEncoder,
                                   StandardScaler)
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm_missing.csv', sep=';')

In [3]:
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Response', axis=1), 
    data['Response'], 
    test_size=0.3,
    stratify=data['Response'],
    random_state=42)

In [4]:
# создаем собственный класс, выполняющий скорректированное 
# логарифмическое преобразование
class CorrLogTransformer(BaseEstimator, TransformerMixin):
    """
    Приравнивает значение признака к нижнему пороговому 
    значению - очень маленькому положительному числу, 
    делит его на среднее признака и добавляет значение 
    от 0 до 1 - корректировку асимметрии, берет 
    натуральный логарифм полученного результата.
    
    Параметры:
    lower: float, по умолчанию 0.001
       Нижнее пороговое значение.
    k: float, по умолчанию 0.2
        Корректировка асимметрии.
    copy: bool, по умолчанию True
        Возвращает копию.
    """
    def __init__(self, lower=0.001, k=0.2, copy=True):
        # все параметры для инициализации публичных атрибутов 
        # должны быть заданы в методе __init__
        
        # публичные атрибуты
        self.lower = lower
        self.k = k
        self.copy = copy
                
    def fit(self, X, y=None):
        # fit должен принимать в качестве аргументов только X и y
        # даже если ваша модель является моделью машинного обучения 
        # без учителя, вы должны принять аргумент y, это требуется 
        # для совместимости с конвейерами!
        
        # обучение модели осуществляется прямо здесь
        
        # создаем пустой словарь, в котором ключами
        # будут имена/целые числа, а значениями - средние
        self._encoder_dict = {}
        
        # функция isinstance() проверяет, является ли
        # объект датафреймом pandas или нет
        if isinstance(X, pd.DataFrame):
            for col in X.columns:
                # вычисляем среднее и записываем в словарь
                self._encoder_dict[col] = np.mean(X[col])
        else:
            for col in range(X.shape[1]):
                # вычисляем среднее и записываем в словарь
                self._encoder_dict[col] = np.mean(X[:, col])
            
        # fit возвращает self
        return self
    
    def transform(self, X):
        # transform принимает в качестве аргумента только X
        
        # выполняем копирование массива во избежание 
        # предупреждения SettingWithCopyWarning
        # "A value is trying to be set on a copy of a slice 
        # from a DataFrame (Происходит попытка изменить 
        # значение в копии среза данных датафрейма)"
        if self.copy:
            X = X.copy()
        
        # применяем преобразование к X
        if isinstance(X, pd.DataFrame):
            for col in X.columns:
                X[col] = (np.log(np.clip(X[col], self.lower, None) / 
                                 self._encoder_dict[col] + self.k))
        else:    
            for col in range(X.shape[1]):
                X[:, col] = (np.log(np.clip(X[:, col], self.lower, None) / 
                                    self._encoder_dict[col] + self.k))
                           
        return X

In [5]:
# создаем игрушечный обучающий датафрейм pandas
train = pd.DataFrame(
    {'Balance': [8.3, 9.4, 10.2, 0], 
     'Age': [23, 29, 36, 44]})
train

,Balance,Age
0,8.3,23
1,9.4,29
2,10.2,36
3,0.0,44


In [6]:
# создаем игрушечный тестовый датафрейм pandas
test = pd.DataFrame(
    {'Balance': [10.4, 3.1, 22.5, -1], 
     'Age': [13, 19, 66, 33]})
test

,Balance,Age
0,10.4,13
1,3.1,19
2,22.5,66
3,-1.0,33


In [7]:
# смотрим, как будут выглядеть преобразования в игрушечных
# обучающем и тестовом датафреймах pandas
print(np.log((train['Balance'].clip(0.001) / 
              train['Balance'].mean()) + 0.2))
print(np.log((train['Age'].clip(0.001) / 
              train['Age'].mean()) + 0.2))
print("")
print(np.log((test['Balance'].clip(0.001) / 
              train['Balance'].mean()) + 0.2))
print(np.log((test['Age'].clip(0.001) / 
              train['Age'].mean()) + 0.2))

0    0.329278
1    0.436751
2    0.508242
3   -1.608721
Name: Balance, dtype: float64
0   -0.108733
1    0.075838
2    0.255347
3    0.427444
Name: Age, dtype: float64

0    0.525343
1   -0.439367
2    1.231337
3   -1.608721
Name: Balance, dtype: float64
0   -0.520978
1   -0.253915
2    0.788457
3    0.182322
Name: Age, dtype: float64


In [8]:
# создаем экземпляр класса CorrLogTransformer
corrlog = CorrLogTransformer()
# обучаем модель
corrlog.fit(train)
# выполняем преобразование игрушечного
# обучающего датафрейма pandas
train = corrlog.transform(train)
train

,Balance,Age
0,0.329278,-0.108733
1,0.436751,0.075838
2,0.508242,0.255347
3,-1.608721,0.427444


In [9]:
# выполняем преобразование игрушечного
# тестового датафрейма pandas
test = corrlog.transform(test)
test

,Balance,Age
0,0.525343,-0.520978
1,-0.439367,-0.253915
2,1.231337,0.788457
3,-1.608721,0.182322


In [10]:
# создаем игрушечный обучающий датафрейм pandas
train = pd.DataFrame(
    {'Balance': [8.3, 9.4, 10.2, 0], 
     'Age': [23, 29, 36, 44]})
# создаем экземпляр класса, отключив копирование
corrlog = CorrLogTransformer(copy=False)
# обучаем модель
corrlog.fit(train[['Age']])
# применяем модель
train['Age'] = corrlog.transform(train[['Age']])
train

/var/folders/x9/w1kvj6ms0p52l8j6crt19vmh0000gn/T/ipykernel_26455/2534646667.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = (np.log(np.clip(X[col], self.lower, None) /


,Balance,Age
0,8.3,-0.108733
1,9.4,0.075838
2,10.2,0.255347
3,0.0,0.427444


In [11]:
# создаем игрушечный обучающий датафрейм pandas
train = pd.DataFrame(
    {'Balance': [8.3, 9.4, 10.2, 0], 
     'Age': [23, 29, 36, 44]})
# создаем экземпляр класса, включив копирование
corrlog = CorrLogTransformer(copy=True)
# обучаем модель
corrlog.fit(train[['Age']])
# применяем модель
train['Age'] = corrlog.transform(train[['Age']])
train

,Balance,Age
0,8.3,-0.108733
1,9.4,0.075838
2,10.2,0.255347
3,0.0,0.427444


In [12]:
# создаем игрушечный обучающий массив NumPy
np_train = np.array(pd.DataFrame(
    {'Balance': [8.3, 9.4, 10.2, 0], 
     'Age': [23, 29, 36, 44]}))
np_train

array([[ 8.3, 23. ],
       [ 9.4, 29. ],
       [10.2, 36. ],
       [ 0. , 44. ]])

In [13]:
# создаем игрушечный тестовый массив NumPy
np_test = np.array(pd.DataFrame(
    {'Balance': [10.4, 3.1, 22.5, -1], 
     'Age': [13, 19, 66, 33]}))
np_test

array([[10.4, 13. ],
       [ 3.1, 19. ],
       [22.5, 66. ],
       [-1. , 33. ]])

In [14]:
# обучаем модель
corrlog.fit(np_train)
# выполняем преобразование игрушечного
# обучающего массива NumPy
np_train = corrlog.transform(np_train)
np_train

array([[ 0.32927796, -0.1087332 ],
       [ 0.43675074,  0.07583808],
       [ 0.50824164,  0.25534669],
       [-1.60872132,  0.42744401]])

In [15]:
# выполняем преобразование игрушечного
# тестового массива NumPy
np_test = corrlog.transform(np_test)
np_test

array([[ 0.52534339, -0.520978  ],
       [-0.43936666, -0.25391521],
       [ 1.2313369 ,  0.78845736],
       [-1.60872132,  0.18232156]])

In [16]:
# создаем собственный класс, заменяющий отрицательные
# и нулевые значения на небольшие положительные
class Replacer(BaseEstimator, TransformerMixin):
    """
    Заменяет отрицательные и нулевые значения на
    небольшое положительное значение.
    
    Параметры
    ----------
    repl: float, по умолчанию 0.1
        Значение для замены.
    """
    def __init__(self, repl_value=0.1):
        self.repl_value = repl_value
    
    # fit здесь бездельничает
    def fit(self, X, y=None):
        return self
    
    # transform выполняет всю работу: применяет преобразование 
    # с помощью заданного значения параметра repl_value
    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            X[X <= 0] = self.repl_value
        else:
            X = np.where(X <= 0, self.repl_value, X)
        return X

In [17]:
# создаем игрушечный обучающий датафрейм pandas
train = pd.DataFrame(
    {'Balance': [0, 9.4, 10.2, 0], 
     'Income': [23, 29, -1, 44]})
train

,Balance,Income
0,0.0,23
1,9.4,29
2,10.2,-1
3,0.0,44


In [18]:
# создаем игрушечный тестовый датафрейм pandas
test = pd.DataFrame(
    {'Balance': [-2, 3.1, 22.5, -1], 
     'Income': [13, -2, 0, 33]})
test

,Balance,Income
0,-2.0,13
1,3.1,-2
2,22.5,0
3,-1.0,33


In [19]:
# создаем экземпляр класса Replacer
repl = Replacer(repl_value=0.1)
# обучаем модель
repl.fit(train)
# выполняем замены в игрушечном
# обучающем датафрейме pandas
train = repl.transform(train)
train

,Balance,Income
0,0.1,23.0
1,9.4,29.0
2,10.2,0.1
3,0.1,44.0


In [20]:
# выполняем замены в игрушечном
# тестовом датафрейме pandas
test = repl.transform(test)
test

,Balance,Income
0,0.1,13.0
1,3.1,0.1
2,22.5,0.1
3,0.1,33.0


In [21]:
# создаем игрушечный обучающий массив NumPy
np_train = np.array(pd.DataFrame(
    {'Balance': [0, 9.4, 10.2, 0], 
     'Income': [23, 29, -1, 44]}))
np_train

array([[ 0. , 23. ],
       [ 9.4, 29. ],
       [10.2, -1. ],
       [ 0. , 44. ]])

In [22]:
# создаем игрушечный тестовый массив NumPy
np_test = np.array(pd.DataFrame(
    {'Balance': [-2, 3.1, 22.5, -1], 
     'Income': [13, -2, 0, 33]}))
np_test

array([[-2. , 13. ],
       [ 3.1, -2. ],
       [22.5,  0. ],
       [-1. , 33. ]])

In [23]:
# обучаем модель
repl.fit(np_train)
# выполняем  замены в игрушечном
# обучающем массиве NumPy
np_train = repl.transform(np_train)
np_train

array([[ 0.1, 23. ],
       [ 9.4, 29. ],
       [10.2,  0.1],
       [ 0.1, 44. ]])

In [24]:
# выполняем замены в игрушечном
# тестовом массиве NumPy
np_test = repl.transform(np_test)
np_test

array([[ 0.1, 13. ],
       [ 3.1,  0.1],
       [22.5,  0.1],
       [ 0.1, 33. ]])

In [25]:
# создаем собственный класс, выполняющий биннинг
class CustomDiscretizer(BaseEstimator, TransformerMixin):
    """
    Выполняет биннинг количественных признаков.
    
    Параметры
    ----------
    bins: list
        Список бинов.
    """
    def __init__(self, bins=[-np.inf, 2, 4, np.inf]):
        self.bins = bins
    
    # fit опять бездельничает
    def fit(self, X, y=None):  
        return self
    
    # transform выполняет всю работу: применяет
    # преобразование с помощью заданного
    # значения параметра bins
    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            cols = X.columns.tolist()
            X = pd.DataFrame(data=np.digitize(X, self.bins), 
                             columns=cols)
        else:    
            X = np.digitize(X, self.bins)
        return X

In [26]:
# создаем игрушечный обучающий датафрейм pandas
train = pd.DataFrame(
    {'Balance': [0, 1.4, 2.2, 5.5, 4.3], 
     'Income': [3, 5, 1, 4, 6]})
train

,Balance,Income
0,0.0,3
1,1.4,5
2,2.2,1
3,5.5,4
4,4.3,6


In [27]:
# создаем экземпляр класса CustomDiscretizer
disc = CustomDiscretizer()
# обучаем модель
disc.fit(train)
# выполняем биннинг в игрушечном
# обучающем датафрейме pandas
train = disc.transform(train)
train

,Balance,Income
0,1,2
1,1,3
2,2,1
3,3,3
4,3,3


In [28]:
# создаем игрушечный обучающий массив NumPy
np_train = np.array(pd.DataFrame(
    {'Balance': [0, 1.4, 2.2, 5.5, 4.3], 
     'Income': [3, 5, 1, 4, 6]}))
np_train

array([[0. , 3. ],
       [1.4, 5. ],
       [2.2, 1. ],
       [5.5, 4. ],
       [4.3, 6. ]])

In [29]:
# обучаем модель
disc.fit(np_train)
# выполняем  биннинг в игрушечном
# обучающем массиве NumPy
np_train = disc.transform(np_train)
np_train

array([[1, 2],
       [1, 3],
       [2, 1],
       [3, 3],
       [3, 3]])

In [30]:
# создаем собственный класс, выполняющий стандартизацию
class CustomScaler(BaseEstimator, TransformerMixin):
    """
    Выполняет стандартизацию признаков.
    
    Параметры
    ----------
    strategy: str, по умолчанию 'standard_scaler'
        Стратегия стандартизации.
    corr: bool, по умолчанию False
        Корректировка по Гельману.
    copy: bool, по умолчанию True
         Выполняет копирование датафрейма.
    """
    def __init__(self, strategy='standard_scaler', 
                 copy=True, corr=False):
        self.strategy = strategy
        self.corr = corr
        self.copy = copy
        
        if corr and strategy == 'minmax_scaler':
            raise ValueError('Corr=True только для ' + 
                             'стратегии standard_scaler')
    
    # частный метод, который с помощью функции isinstance()
    # проверяет, является ли наш объект массивом NumPy
    def __is_numpy(self, X):
        return isinstance(X, np.ndarray)    
    
    def fit(self, X, y=None):
        self.dict = {}
        
        # если 1D-массив, то переводим в 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
             
        # записываем результат __is_numpy()
        is_np = self.__is_numpy(X)
        
        # записываем количество столбцов
        ncols = X.shape[1]
        
        if self.strategy == 'standard_scaler':
            # если объект - массив NumPy, 
            # выполняем следующие действия:
            if is_np:       
                # по каждому столбцу массива NumPy
                for col in range(ncols):
                    # вычисляем среднее
                    mean = np.mean(X[:, col])
                    # вычисляем стандартное отклонение
                    std = np.std(X[:, col], ddof=1)
                    if self.corr:
                        std = np.std(X[:, col], ddof=1) * 2
                    self.dict[col] = (mean, std)
            
            # если объект - датафрейм pandas, 
            # выполняем следующие действия:         
            else:
                # по каждому столбцу датафрейма pandas
                for col in X.columns:
                    # вычисляем среднее
                    mean = X[col].mean()
                    # вычисляем стандартное отклонение
                    std = X[col].std()
                    if self.corr:
                        std = X[col].std() * 2
                    self.dict[col] = (mean, std)
                        
        if self.strategy == 'minmax_scaler':
            # если объект - массив NumPy, 
            # выполняем следующие действия:
            if is_np:
                # по каждому столбцу массива NumPy
                for col in range(ncols):
                    # вычисляем минимальное значение
                    min_value = np.min(X[:, col])
                    # вычисляем разницу между максимальным
                    # и минимальным значениями
                    rng = np.ptp(X[:, col])
                    self.dict[col] = (min_value, rng)
                    
            # если объект - датафрейм pandas, 
            # выполняем следующие действия: 
            else:
                # по каждому столбцу датафрейма pandas
                for col in X.columns:
                    # вычисляем среднее
                    min_value = X[col].min()
                    # вычисляем разницу между максимальным
                    # и минимальным значениями
                    rng = X[col].max() - X[col].min()
                    self.dict[col] = (min_value, rng)        
                         
        return self
    
    def transform(self, X):
        
        if self.copy:
            X = X.copy()
            
        # если 1D-массив, то переводим в 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
        
        # записываем результат __is_numpy()
        is_np = self.__is_numpy(X)
        
        # записываем количество столбцов
        ncols = X.shape[1]
    
        # если объект - массив NumPy, 
        # выполняем следующие действия:
        if is_np:
            # по каждому столбцу массива NumPy
            for col in range(ncols):
                # выполняем стандартизацию
                X[:, col] = ((X[:, col] - self.dict[col][0]) / 
                             self.dict[col][1])
        
        # если объект - датафрейм pandas, 
        # выполняем следующие действия:     
        else:
            # по каждому столбцу датафрейма pandas
            for col in X.columns:
                # выполняем стандартизацию
                X[col] = ((X[col] - self.dict[col][0]) / 
                          self.dict[col][1])
                
        return X

In [31]:
# создаем игрушечный обучающий датафрейм pandas
toy_train = pd.DataFrame(
    {'Balance': [8.3, 9.4, 10.2, 0], 
     'Age': [23, 29, 36, 44]})

# создаем игрушечный тестовый датафрейм pandas
toy_test = pd.DataFrame(
    {'Balance': [10.4, 3.1, 22.5, -1], 
     'Age': [13, 19, 66, 33]})

In [32]:
# смотрим, как должны выглядеть результаты
balance_mean = toy_train['Balance'].mean()
balance_std = toy_train['Balance'].std()
age_mean = toy_train['Age'].mean()
age_std = toy_train['Age'].std()
print((toy_train['Balance'] - balance_mean) / balance_std)
print((toy_train['Age'] - age_mean) / age_std)
print("")
print((toy_test['Balance'] - balance_mean) / balance_std)
print((toy_test['Age'] - age_mean) / age_std)

0    0.281031
1    0.514340
2    0.684019
3   -1.479390
Name: Balance, dtype: float64
0   -1.104315
1   -0.441726
2    0.331295
3    1.214747
Name: Age, dtype: float64

0    0.726439
1   -0.821883
2    3.292835
3   -1.691489
Name: Balance, dtype: float64
0   -2.208631
1   -1.546041
2    3.644240
3    0.000000
Name: Age, dtype: float64


In [33]:
# применяем класс к датафреймам pandas
scaler = CustomScaler(strategy='standard_scaler', 
                      corr=False)
scaler.fit(toy_train)
toy_train = scaler.transform(toy_train)
toy_test = scaler.transform(toy_test)

In [34]:
# смотрим результат в игрушечном 
# обучающем датафрейме
toy_train

,Balance,Age
0,0.281031,-1.104315
1,0.514340,-0.441726
2,0.684019,0.331295
3,-1.479390,1.214747


In [35]:
# смотрим результат в игрушечном 
# тестовом датафрейме
toy_test

,Balance,Age
0,0.726439,-2.208631
1,-0.821883,-1.546041
2,3.292835,3.644240
3,-1.691489,0.000000


In [36]:
# взглянем на стандартные отклонения
for col in toy_train.columns:
    print(col, toy_train[col].std())

Balance 1.0
Age 1.0


In [37]:
# создаем игрушечный обучающий массив NumPy
np_toy_train = np.array(pd.DataFrame(
    {'Balance': [8.3, 9.4, 10.2, 0], 
     'Age': [23, 29, 36, 44]}))

# создаем игрушечный тестовый массив NumPy
np_toy_test = np.array(pd.DataFrame(
    {'Balance': [10.4, 3.1, 22.5, -1], 
     'Age': [13, 19, 66, 33]}))

In [38]:
# применяем класс к массивам NumPy
scaler.fit(np_toy_train)
np_toy_train = scaler.transform(np_toy_train)
np_toy_test = scaler.transform(np_toy_test)

In [39]:
# смотрим результат в игрушечном 
# обучающем массиве NumPy
np_toy_train

array([[ 0.28103104, -1.10431526],
       [ 0.51433982, -0.4417261 ],
       [ 0.68401894,  0.33129458],
       [-1.4793898 ,  1.21474679]])

In [40]:
# смотрим результат в игрушечном 
# тестовом массиве NumPy
np_toy_test

array([[ 0.72643872, -2.20863052],
       [-0.82188322, -1.54604137],
       [ 3.29283537,  3.64424036],
       [-1.6914887 ,  0.        ]])

In [41]:
# создаем игрушечный обучающий датафрейм pandas
toy_train = pd.DataFrame(
    {'Balance': [8.3, 9.4, 10.2, 0], 
     'Age': [23, 29, 36, 44]})

# создаем игрушечный тестовый датафрейм pandas
toy_test = pd.DataFrame(
    {'Balance': [10.4, 3.1, 22.5, -1], 
     'Age': [13, 19, 66, 33]})

In [42]:
# смотрим, как должны выглядеть результаты
balance_mean = toy_train['Balance'].mean()
balance_2std = toy_train['Balance'].std() * 2
age_mean = toy_train['Age'].mean()
age_2std = toy_train['Age'].std() * 2

print((toy_train['Balance'] - balance_mean) / balance_2std)
print((toy_train['Age'] - age_mean) / age_2std)
print("")
print((toy_test['Balance'] - balance_mean) / balance_2std)
print((toy_test['Age'] - age_mean) / age_2std)

0    0.140516
1    0.257170
2    0.342009
3   -0.739695
Name: Balance, dtype: float64
0   -0.552158
1   -0.220863
2    0.165647
3    0.607373
Name: Age, dtype: float64

0    0.363219
1   -0.410942
2    1.646418
3   -0.845744
Name: Balance, dtype: float64
0   -1.104315
1   -0.773021
2    1.822120
3    0.000000
Name: Age, dtype: float64


In [43]:
# применяем класс к датафреймам pandas
scaler = CustomScaler(strategy='standard_scaler', 
                      corr=True)
scaler.fit(toy_train)
toy_train = scaler.transform(toy_train)
toy_test = scaler.transform(toy_test)

In [44]:
# смотрим результат в игрушечном 
# обучающем датафрейме
toy_train

,Balance,Age
0,0.140516,-0.552158
1,0.257170,-0.220863
2,0.342009,0.165647
3,-0.739695,0.607373


In [45]:
# смотрим результат на игрушечном тестовом датафрейме
toy_test

,Balance,Age
0,0.363219,-1.104315
1,-0.410942,-0.773021
2,1.646418,1.822120
3,-0.845744,0.000000


In [46]:
# взглянем на стандартные отклонения
for col in toy_train.columns:
    print(col, toy_train[col].std())

Balance 0.5
Age 0.5


In [47]:
# создаем игрушечный обучающий массив NumPy
np_toy_train = np.array(pd.DataFrame(
    {'Balance': [8.3, 9.4, 10.2, 0], 
     'Age': [23, 29, 36, 44]}))

# создаем игрушечный тестовый массив NumPy
np_toy_test = np.array(pd.DataFrame(
    {'Balance': [10.4, 3.1, 22.5, -1], 
     'Age': [13, 19, 66, 33]}))

In [48]:
# сприменяем класс к массивам NumPy
scaler.fit(np_toy_train)
np_toy_train = scaler.transform(np_toy_train)
np_toy_test = scaler.transform(np_toy_test)

In [49]:
# смотрим результат в игрушечном 
# обучающем массиве NumPy
np_toy_train

array([[ 0.14051552, -0.55215763],
       [ 0.25716991, -0.22086305],
       [ 0.34200947,  0.16564729],
       [-0.7396949 ,  0.60737339]])

In [50]:
# смотрим результат в игрушечном 
# тестовом массиве NumPy
np_toy_test

array([[ 0.36321936, -1.10431526],
       [-0.41094161, -0.77302068],
       [ 1.64641768,  1.82212018],
       [-0.84574435,  0.        ]])

In [51]:
# создаем игрушечный обучающий датафрейм pandas
toy_train = pd.DataFrame(
    {'Balance': [8.3, 9.4, 10.2, 0], 
     'Age': [23, 29, 36, 44]})

# создаем игрушечный тестовый датафрейм pandas
toy_test = pd.DataFrame(
    {'Balance': [10.4, 3.1, 22.5, -1], 
     'Age': [13, 19, 66, 33]})

In [52]:
# смотрим, как должны выглядеть результаты
balance_min = toy_train['Balance'].min()
balance_range = (toy_train['Balance'].max() - 
                 toy_train['Balance'].min())
age_min = toy_train['Age'].min()
age_range = (toy_train['Age'].max() - 
             toy_train['Age'].min())
print((toy_train['Balance'] - balance_min) / balance_range)
print((toy_train['Age'] - age_min) / age_range)
print("")
print((toy_test['Balance'] - balance_min) / balance_range)
print((toy_test['Age'] - age_min) / age_range)

0    0.813725
1    0.921569
2    1.000000
3    0.000000
Name: Balance, dtype: float64
0    0.000000
1    0.285714
2    0.619048
3    1.000000
Name: Age, dtype: float64

0    1.019608
1    0.303922
2    2.205882
3   -0.098039
Name: Balance, dtype: float64
0   -0.476190
1   -0.190476
2    2.047619
3    0.476190
Name: Age, dtype: float64


In [53]:
# применяем класс к датафреймам pandas
scaler = CustomScaler(strategy='minmax_scaler', 
                      corr=False)
scaler.fit(toy_train)
toy_train = scaler.transform(toy_train)
toy_test = scaler.transform(toy_test)

In [54]:
# смотрим результат в игрушечном 
# обучающем датафрейме
toy_train

,Balance,Age
0,0.813725,0.000000
1,0.921569,0.285714
2,1.000000,0.619048
3,0.000000,1.000000


In [55]:
# смотрим результат в игрушечном 
# тестовом датафрейме
toy_test

,Balance,Age
0,1.019608,-0.476190
1,0.303922,-0.190476
2,2.205882,2.047619
3,-0.098039,0.476190


In [56]:
# создаем игрушечный обучающий массив NumPy
np_toy_train = np.array(pd.DataFrame(
    {'Balance': [8.3, 9.4, 10.2, 0], 
     'Age': [23, 29, 36, 44]}))

# создаем игрушечный тестовый массив NumPy
np_toy_test = np.array(pd.DataFrame(
    {'Balance': [10.4, 3.1, 22.5, -1], 
     'Age': [13, 19, 66, 33]}))

In [57]:
# сприменяем класс к массивам NumPy
scaler.fit(np_toy_train)
np_toy_train = scaler.transform(np_toy_train)
np_toy_test = scaler.transform(np_toy_test)

In [58]:
# смотрим результат в игрушечном 
# обучающем массиве NumPy
np_toy_train

array([[0.81372549, 0.        ],
       [0.92156863, 0.28571429],
       [1.        , 0.61904762],
       [0.        , 1.        ]])

In [59]:
# смотрим результат в игрушечном 
# тестовом массиве NumPy
np_toy_test

array([[ 1.01960784, -0.47619048],
       [ 0.30392157, -0.19047619],
       [ 2.20588235,  2.04761905],
       [-0.09803922,  0.47619048]])

In [60]:
# создаем список категориальных признаков
cat_columns = X_train.select_dtypes(
    include='object').columns.tolist()
# создаем список количественных признаков
num_columns = X_train.select_dtypes(
    exclude='object').columns.tolist()
# создаем одноэлементный список для
# признака Customer LifeTime Value
clv_corrlog = ['Customer Lifetime Value']
# создаем одноэлементный список для
# признака Number of Policies
numberofpol_bin = ['Number of Policies']
# создаем одноэлементный список для 
# признака Monthly Premium Auto
monthly_bin = ['Monthly Premium Auto']

In [61]:
# удаляем из списка количественных признаков признаки Customer 
# LifeTime Value, Number of Policies, Monthly Premium Auto
num_columns = list(set(num_columns).difference(
    set(clv_corrlog + numberofpol_bin + monthly_bin)))
num_columns

['Income',
 'Months Since Last Claim',
 'Months Since Policy Inception',
 'Number of Open Complaints']

In [62]:
# создаем конвейер для количественных признаков 
# Number of Open Complaints, Income, 
# Months Since Last Claim и 
# Months Since Policy Inception
num_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='mean')),
    ('repl', Replacer()),
    ('box_cox', PowerTransformer(method='box-cox', 
                                 standardize=True))
])

# создаем конвейер для количественного 
# признака Customer Lifetime Value
clv_corrlog_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='mean')),
    ('corrlog', CorrLogTransformer(k=0.001)),
    ('scaler', CustomScaler())
])

# создаем конвейер для количественного
# признака Number of Policies
numberofpol_bin_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='most_frequent')),
    ('binn', CustomDiscretizer()),
    ('ohe', OneHotEncoder(sparse=False, 
                          handle_unknown='ignore'))
])

# создаем конвейер для количественного 
# признака Monthly Premium Auto
monthly_bin_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='most_frequent')),
    ('binn', CustomDiscretizer()),
    ('ohe', OneHotEncoder(sparse=False, 
                          handle_unknown='ignore'))
])

# создаем конвейер для категориальных признаков
cat_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, 
                          handle_unknown='ignore'))
])

In [63]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [
    ('num', num_pipe, num_columns),
    ('clv_corrlog', clv_corrlog_pipe, clv_corrlog),
    ('numberofpol_bin', numberofpol_bin_pipe, numberofpol_bin),
    ('monthly_bin', monthly_bin_pipe, monthly_bin),
    ('cat', cat_pipe, cat_columns)
]

# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)
transformer

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imp', SimpleImputer()),
                                                 ('repl', Replacer()),
                                                 ('box_cox',
                                                  PowerTransformer(method='box-cox'))]),
                                 ['Income', 'Months Since Last Claim',
                                  'Months Since Policy Inception',
                                  'Number of Open Complaints']),
                                ('clv_corrlog',
                                 Pipeline(steps=[('imp', SimpleImputer()),
                                                 ('corrlog',
                                                  CorrLogTransformer(k=0.001)),
                                                 ('scaler', CustomSc...
                                 Pipeline(steps=[('imp',
                                                  SimpleImputer(stra

In [64]:
# задаем итоговый конвейер
ml_pipe = Pipeline([
    ('tr', transformer), 
    ('lr', LogisticRegression(solver='lbfgs', 
                              max_iter=200))
])

In [65]:
# задаем сетку гиперпараметров
param_grid = {
    'tr__num__repl__repl_value': [0.005, 0.01, 0.2],
    'tr__clv_corrlog__corrlog__k': [0.1, 0.5, 0.6, 0.7, 0.8],
    'tr__numberofpol_bin__binn__bins': [[-np.inf, 2, 4, 6, np.inf], 
                                        [-np.inf, 3, 6, np.inf]],
    'tr__monthly_bin__binn__bins': [[-np.inf, 100, 150, 200, 250, np.inf],
                                    [-np.inf, 100, 150, 200, np.inf]], 
    'lr__C': [0.1, 0.2, 0.3]
}

# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров, оптимизируемую метрику качества 
# и указав количество блоков перекрестной проверки
gs = GridSearchCV(ml_pipe, 
                  param_grid, 
                  scoring='roc_auc', 
                  cv=5)

# выполняем поиск по сетке
gs.fit(X_train, y_train)

# смотрим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров:\n{}".format(
    gs.best_params_))
# смотрим наилучшее значение AUC
print("Наилучшее значение AUC: {:.3f}".format(
    gs.best_score_))
# смотрим значение AUC на тестовой выборке
print("AUC на тестовой выборке: {:.3f}".format(
    roc_auc_score(y_test, gs.predict_proba(X_test)[:, 1])))

Наилучшие значения гиперпараметров:
{'lr__C': 0.2, 'tr__clv_corrlog__corrlog__k': 0.6, 'tr__monthly_bin__binn__bins': [-inf, 100, 150, 200, inf], 'tr__num__repl__repl_value': 0.005, 'tr__numberofpol_bin__binn__bins': [-inf, 3, 6, inf]}
Наилучшее значение AUC: 0.668
AUC на тестовой выборке: 0.669
